<a href="https://colab.research.google.com/github/bhowad-akash/Machine-Learning/blob/main/Convolutional_Neural_Network_Architectures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Typical CNN Architectures**

Below-mentioned are a few well-known CNN architectures (for image
classification, but can be adapted for other applications)


*   AlexNet
*   GoogLeNet (a.k.a. InceptionNet)
*   VGGNet
*   ResNet
*   Xception

They often involve new ideas for training, and new modules that can
be reused in new architectures.

Training these models can take a large amount of resources.

Keras supports typical models with pre-trained weights on ImageNet
E.g. Xception, VGG16, VGG19, ResNet, ResNetV2, InceptionV3, etc.

Refer to the https://keras.io/applications/ for more details.
These are for classifying images (into 1,000 categories)

In order to classify the other images, these Pre trained CNN models could be used.

## ***Image Classification using Pre-Trained Networks***

In [27]:
#Import necessary libraries
import tensorflow as tf
import keras

In [28]:
#Loading Pre-Trained Resnet50 Model
model = keras.applications.ResNet50(weights='imagenet')

102967424/102967424 [==============================] - 1s 0us/step


In [29]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                       

In [30]:
#Importing Keras module that contains utility functions for handling images
import keras.utils as image

#Mounting the drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
'''The function to load image from the specified path
The image is resized to 224x224 pixels, which is the input size required by the ResNet50 model'''

img = image.load_img("/content/drive/MyDrive/Applied Machine Learning/Week 8/IndianElephant.jpg", target_size=(224, 224))

In [32]:
#Converting image to Numpy array (common format for image data in deep learning)
img = image.img_to_array(img)
img.shape

(224, 224, 3)

In [33]:
#Reshaping data for the model

'''The ResNet50 model, like many other convolutional neural networks (CNNs),
expects the input to be in the form of a 4D tensor with the shape (batch_size, height, width, channels)

After loading and converting the image to a NumPy array,
the shape is (height, width, channels), which in this case would be (224, 224, 3).

The model expects a shape of (batch_size, height, width, channels).
For a single image, the batch size is 1, so the expected shape is (1, 224, 224, 3)'''

img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))
img.shape

(1, 224, 224, 3)

In [34]:
#Pre-processing the image for the model

'''In order to match the format expected by the ResNet50 model expects,
we need to pereform the pre processing step.

This step esentially involves scaling pixel values and
performing any other model-specific preprocessing.'''

img = keras.applications.resnet50.preprocess_input(img)

(1, 224, 224, 3)

In [35]:
#Performing Predictions

'''Uses the pre-trained model to predict the class probabilities for the input image.
Y_prob is an array of probabilities for each class.'''

Y_prob = model.predict(img)

1/1 [==============================] - 2s 2s/step


In [36]:
#Top K Predictions

'''Converts the model's output probabilities into human-readable class labels
and their associated probabilities.'''

top_k = keras.applications.resnet50.decode_predictions(Y_prob, top=3)

35363/35363 [==============================] - 0s 0us/step


In [42]:
'''Iterates through the top 3 predictions and prints the class ID,
class name, and the probability (converted to a percentage)'''

for class_id, name, y_proba in top_k[0]:
  print("class_id:", class_id, "name:", name, " ", y_proba*100, "%")

class_id: n02504458 name: African_elephant   61.43898963928223 %
class_id: n01871265 name: tusker   27.4200439453125 %
class_id: n02504013 name: Indian_elephant   10.928305238485336 %
